In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam, Adadelta, RMSprop, Adagrad, Nadam, Ftrl
from keras.regularizers import l2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [2]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [3]:
X = X.drop('sub_area', axis=1)
X_test = X_test.drop('sub_area', axis=1)
used.append('Removed sub_area')

In [4]:
# X = pd.get_dummies(X)
# X_test = pd.get_dummies(X_test) 
# used.append('OneHot Encoding')

In [5]:
from sklearn.calibration import LabelEncoder

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
print("Train: Categorical columns:", categorical_columns)

label_encoder = LabelEncoder()

for column in categorical_columns:
    X[column] = label_encoder.fit_transform(X[column])

categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
print("Test: Categorical columns:", categorical_columns_test)

label_encoder = LabelEncoder()

for column in categorical_columns_test:
    X_test[column] = label_encoder.fit_transform(X_test[column])

used.append('Label Encoding')

Train: Categorical columns: ['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']
Test: Categorical columns: ['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']


In [6]:
# # drop all columns in X_train with dtypes object
# for col in X.columns:
#     if X[col].dtype == 'object':
#         X.drop(col, axis=1, inplace=True)

# # drop all columns in X_test with dtypes object
# for col in X_test.columns:
#     if X_test[col].dtype == 'object':
#         X_test.drop(col, axis=1, inplace=True)

# used.append("Removed Object Dtypes")

In [7]:
# X = X[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
#  'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
#  'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 'big_market_km', 'public_healthcare_km', 'workplaces_km']]
# X_test = X_test[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
#  'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
#  'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 'big_market_km', 'public_healthcare_km', 'workplaces_km']]

# used.append("Forward Feature Selection (n=14)")

In [8]:
# from sklearn.feature_selection import VarianceThreshold


# threshold_value = 4000000
# variance_filter = VarianceThreshold(threshold=threshold_value)

# X = variance_filter.fit_transform(X)
# X_test = variance_filter.transform(X_test)

# used.append("Variance Based Feature Selection")

In [9]:
# import statsmodels.api as sm

# # Add a constant term to the feature matrix
# X_with_const = sm.add_constant(X)

# # Fit a linear regression model
# model = sm.OLS(y, X_with_const).fit()

# # Get p-values for each feature
# p_values = model.pvalues[1:]  # Exclude the constant term

# # Set your desired threshold for p-value
# threshold = 0.00001

# # Filter features based on p-value
# selected_features = p_values[p_values < threshold].index

# # Display selected features
# print("Selected Features:")
# print(selected_features)
# print(len(selected_features))

# # Select columns in the DataFrame
# X = X[selected_features]
# X_test = X_test[selected_features]

# used.append("P-Value Based Feature Selection")

In [10]:
# Build the Random Forest model
from catboost import CatBoostRegressor
rf_model = CatBoostRegressor(iterations=1000, depth=10,learning_rate=0.1,  random_seed=42, loss_function='RMSE')
used.append('CatBoostRegressor(iterations=1000, depth=10,learning_rate=0.1,  random_seed=42, loss_function=RMSE)')

# Train the model to get feature importances
rf_model.fit(X, y)

# Feature importance analysis
feature_importances = rf_model.feature_importances_

# Create a DataFrame with feature names and their importance scores
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the features based on importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Select the top N most important features
top_n_features = 20  # You can choose the number of top features based on your preference
selected_features = feature_importance_df.head(top_n_features)['Feature'].tolist()

# Subset the original data to include only the selected features
X = X[selected_features]
X_test = X_test[selected_features]

used.append("CatBoostRegressor Feature Importance Based Feature Selection")

0:	learn: 20449622.9594865	total: 460ms	remaining: 7m 39s
1:	learn: 19257762.9099224	total: 930ms	remaining: 7m 43s
2:	learn: 18231040.9135492	total: 1.34s	remaining: 7m 26s
3:	learn: 17346758.9795710	total: 1.64s	remaining: 6m 48s
4:	learn: 16588711.6432465	total: 2.06s	remaining: 6m 49s
5:	learn: 15943238.7725489	total: 2.37s	remaining: 6m 33s
6:	learn: 15401089.5216433	total: 2.71s	remaining: 6m 25s
7:	learn: 14944619.5536112	total: 3.02s	remaining: 6m 14s
8:	learn: 14552822.6237068	total: 3.32s	remaining: 6m 5s
9:	learn: 14227511.4460951	total: 3.59s	remaining: 5m 55s
10:	learn: 13959496.7615902	total: 3.87s	remaining: 5m 48s
11:	learn: 13736820.0533851	total: 4.2s	remaining: 5m 45s
12:	learn: 13548854.3518524	total: 4.51s	remaining: 5m 42s
13:	learn: 13391992.6932910	total: 4.86s	remaining: 5m 42s
14:	learn: 13259575.2093402	total: 5.22s	remaining: 5m 42s
15:	learn: 13148022.2832268	total: 5.53s	remaining: 5m 39s
16:	learn: 13055890.2271903	total: 5.84s	remaining: 5m 37s
17:	learn

In [11]:
# pca = PCA(n_components=10)
# principalComponents = pca.fit_transform(X)
# X = pd.DataFrame(data = principalComponents)

# pca2 = PCA(n_components=10)
# principalComponents = pca2.fit_transform(X_test)
# X_test = pd.DataFrame(data = principalComponents)

# used.append('PCA (n=10)')

In [12]:
X = X.astype('float32')
X_test = X_test.astype('float32')
used.append("Converted All Columns To float32")

In [13]:
X = np.log1p(X)
X_test = np.log1p(X_test)
used.append('log Normalization')

In [14]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

In [16]:
X_train.shape

(127054, 20)

In [17]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2)#, interaction_only=True)
# X_train = poly.fit_transform(X_train)
# X_test = poly.fit_transform(X_test)
# used.append('PolynomialFeatures W/O Interaction')
# # used.append('PolynomialFeatures With Interaction')

In [18]:
# # save X_train to csv
# X_train.to_csv('train with poly w/o int.2.csv', index=False)
# X_test.to_csv('test with poly w/o int.csv', index=False)

In [19]:
# pca = PCA(n_components=200)
# principalComponents = pca.fit_transform(X_train)
# X_train = pd.DataFrame(data = principalComponents)

# pca2 = PCA(n_components=200)
# principalComponents = pca2.fit_transform(X_test)
# X_test = pd.DataFrame(data = principalComponents)

# used.append('PCA (n=200)')

In [20]:
features_used = X_train.shape[1]

In [21]:
# Build the Random Forest model
from catboost import CatBoostRegressor
rf_model = CatBoostRegressor(iterations=1000, depth=10,learning_rate=0.1,  random_seed=42, loss_function='RMSE')
used.append('CatBoostRegressor(iterations=1000, depth=10,learning_rate=0.1,  random_seed=42, loss_function=RMSE)')

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
val_predictions = rf_model.predict(X_val)

# Evaluate the model on the validation set
val_mse = mean_squared_error(y_val, val_predictions)
print(f'Validation Mean Squared Error: {val_mse}')

# Make predictions on the test set
predictions = rf_model.predict(X_test)


0:	learn: 20449949.8923506	total: 28.3ms	remaining: 28.2s
1:	learn: 19256325.3489715	total: 55.9ms	remaining: 27.9s
2:	learn: 18247246.0662315	total: 84.2ms	remaining: 28s
3:	learn: 17356275.0523518	total: 115ms	remaining: 28.6s
4:	learn: 16601574.2633341	total: 145ms	remaining: 28.9s
5:	learn: 15953618.4637229	total: 179ms	remaining: 29.6s
6:	learn: 15412070.1293074	total: 211ms	remaining: 29.9s
7:	learn: 14955991.0655545	total: 244ms	remaining: 30.3s
8:	learn: 14570820.7184251	total: 314ms	remaining: 34.6s
9:	learn: 14244521.8244572	total: 351ms	remaining: 34.8s
10:	learn: 13979236.0425280	total: 387ms	remaining: 34.8s
11:	learn: 13758396.5492615	total: 425ms	remaining: 35s
12:	learn: 13571145.4114552	total: 459ms	remaining: 34.8s
13:	learn: 13413737.3919113	total: 498ms	remaining: 35.1s
14:	learn: 13281601.3377415	total: 535ms	remaining: 35.1s
15:	learn: 13174033.5302625	total: 571ms	remaining: 35.1s
16:	learn: 13080856.8123745	total: 608ms	remaining: 35.1s
17:	learn: 12999596.79225

In [22]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day15.4.csv', index=False)

In [23]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 20

Removed sub_area
Label Encoding
CatBoostRegressor(iterations=1000, depth=10,learning_rate=0.1,  random_seed=42, loss_function=RMSE)
CatBoostRegressor Feature Importance Based Feature Selection
Converted All Columns To float32
log Normalization
StandardScaler
CatBoostRegressor(iterations=1000, depth=10,learning_rate=0.1,  random_seed=42, loss_function=RMSE)
